<em style="text-align:center">Copyright Iván Pinar Domínguez</em>

## Importar librerías iniciales e instancia de modelo de chat

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain,TransformChain
f = open('../OpenAI_key.txt')
api_key = f.read()
llm = ChatOpenAI(openai_api_key=api_key)

### Conectar a BD Vectores

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore

In [4]:
embedding_function = OpenAIEmbeddings(openai_api_key=api_key)

In [5]:
vector_store_connection = SKLearnVectorStore(embedding=embedding_function, persist_path="../Bloque 3_Conectores de Datos/ejemplosk_embedding_db", serializer="parquet")

## Cargar cadena QA

In [6]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain #Opción que proporciona también la fuente de datos de la respuesta

In [7]:
chain = load_qa_chain(llm,chain_type='stuff') #chain_type='stuff' se usa cuando se desea una manera simple y directa de cargar y procesar el contenido completo sin dividirlo en fragmentos más pequeños. Es ideal para situaciones donde el volumen de datos no es demasiado grande y se puede manejar de manera eficiente por el modelo de lenguaje en una sola operación.

In [8]:
question = "¿Qué pasó en el siglo de Oro?"

In [9]:
docs = vector_store_connection.similarity_search(question)

In [10]:
chain.run(input_documents=docs,question=question)

C:\Users\Iván Pinar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Durante el Siglo de Oro español, que abarca desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681, hubo un florecimiento en las artes y las letras hispanas. Destacaron figuras como Miguel de Cervantes Saavedra, autor de "Don Quijote de la Mancha", considerada la primera novela moderna de la historia universal. Además, surgieron obras como "Lazarillo de Tormes" y hubo una pléyade de poetas, dramaturgos, prosistas, humanistas y filósofos notables como Quevedo, Góngora, Lope de Vega, Calderón, Tirso de Molina, San Juan de la Cruz, Santa Teresa de Ávila, entre muchos otros.'

### Alternativa con método invoke

In [11]:
#Estructurar un diccionario con los parámetros de entrada
inputs = {
    "input_documents": docs,
    "question": question
}

In [12]:
chain.invoke(inputs)["output_text"]

'En el Siglo de Oro español, que abarca desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681, hubo un florecimiento en las artes y las letras hispanas. Destacaron figuras como Miguel de Cervantes Saavedra, autor de "Don Quijote de la Mancha", considerada la primera novela moderna; así como poetas, dramaturgos, prosistas como Quevedo, Góngora, Lope de Vega, Calderón, entre otros. Además, en este periodo se inició el género de la picaresca con obras como "La vida de Lazarillo de Tormes y de sus fortunas y adversidades".'